In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [10]:
model=load_model('model.h5')

with open('label_encoder_gender', 'rb') as f:
    label_encoder_gender = pickle.load(f)
with open('one_encoder_geo', 'rb') as f:
    label_encoder_geo = pickle.load(f)
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)


In [11]:
input_data = {    'CreditScore': 619,
    'Geography': 'France',
        'Gender':'Male',
    'Age': 42,
    'Tenure': 2,
    'Balance': 0.0,
    'NumOfProducts': 1,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 101348.88
}

In [12]:
# One-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


/Users/sragh/Desktop/GenAI/ANN/annvenv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [13]:
# Create DataFrame from input data
input_df = pd.DataFrame([input_data])
print("Original input data:")
print(input_df)

Original input data:
   CreditScore Geography Gender  Age  Tenure  Balance  NumOfProducts  \
0          619    France   Male   42       2      0.0              1   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1        101348.88  


In [14]:
# Encode Gender
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
print("After gender encoding:")
print(input_df)

After gender encoding:
   CreditScore Geography  Gender  Age  Tenure  Balance  NumOfProducts  \
0          619    France       1   42       2      0.0              1   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1        101348.88  


In [15]:
# Concatenate with one-hot encoded geography
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
print("After concatenating geography encoding:")
print(input_df)

After concatenating geography encoding:
   CreditScore  Gender  Age  Tenure  Balance  NumOfProducts  HasCrCard  \
0          619       1   42       2      0.0              1          1   

   IsActiveMember  EstimatedSalary  Geography_France  Geography_Germany  \
0               1        101348.88               1.0                0.0   

   Geography_Spain  
0              0.0  


In [16]:
# Scale the input data
input_scaled = scaler.transform(input_df)
print("After scaling:")
print(input_scaled)

After scaling:
[[-0.33880827  0.91324755  0.29493847 -1.04241787 -1.21847056 -0.91668767
   0.64920267  0.97481699  0.01595384  1.00150113 -0.57946723 -0.57638802]]


In [17]:
# Make prediction
prediction = model.predict(input_scaled)
prediction_proba = prediction[0][0]

print(f"Prediction probability: {prediction_proba:.4f}")

if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

2025-07-11 17:40:13.512596: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 481ms/step
Prediction probability: 0.0538
The customer is not likely to churn.
